# Analysis of the limitations of the research

In [1]:
import json
import math
import random
import os
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import sys

In [2]:
project_root = os.path.abspath(os.path.join('..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    
print(project_root)
from event_model import event_model as em

c:\Users\aurel\Documents\GitHub\Code_Thesis_GitHub\Code_Thesis_GitHub


In [3]:
dataset={
    'bsphiphi': 'datasets/bsphiphi',
    'minibias': 'datasets/minibias',
}

In [4]:
def load_events_modules_tracks(dataset_path, load_percent, shuffle=False):

    all_events = []
    all_modules = []
    all_tracks = []

    for (dirpath, dirnames, filenames) in os.walk(os.path.abspath(os.path.join('..', dataset[dataset_path]))):
        if shuffle:
            random.shuffle(filenames)

        for i, filename in enumerate(filenames):
            # Get an event
            f = open(os.path.realpath(os.path.join(dirpath, filename)))
            json_data_event = json.loads(f.read())

        ###
            ev = em.event(json_data_event, read_tracks=True)
            md = ev.modules
            tk = ev.real_tracks
        
        ###
            all_events.append(ev) 
            all_modules.append(md)
            all_tracks.append(tk)

            f.close()
        
            if i == int((load_percent * len(filenames))-1):
                break 
        
    print(f'Successfully loaded {len(all_events)} events from {dataset_path} dataset.' )

    return all_events, all_modules, all_tracks

In [5]:
all_events_minibias,all_modules, all_tracks = load_events_modules_tracks('minibias', 1, False)

Successfully loaded 995 events from minibias dataset.


In [6]:
event = all_events_minibias[0]
print(dir(event))
print(event.real_tracks)


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'description', 'hits', 'module_prefix_sum', 'module_zs', 'modules', 'montecarlo', 'number_of_hits', 'number_of_modules', 'real_tracks']
[Track with 5 hits: [#255 module 8 {-25.725, -16.763, -136.918}, #310 module 10 {-15.865, -10.325, -63.08}, #371 module 12 {-12.52, -8.147, -38.08}, #427 module 14 {-9.156, -5.988, -13.08}, #463 module 16 {-5.773, -3.81, 11.919}], Track with 4 hits: [#313 module 10 {-3.351, -37.915, -61.918}, #379 module 12 {-2.479, -29.466, -36.918}, #416 module 14 {-1.87, -21.895, -13.08}, #460 module 16 {-1.174, -13.893, 11.919}], Track with 3 hits: [#526 module 22 {-8.613, 7.776, 88.081}, #565 module 24 {-14.894, 13.435, 111.9

In [7]:
tracks = event.real_tracks
print(tracks, '\n')

track = event.real_tracks[4]
print(track)
print(dir(track))

hits_track = track.hits
print(hits_track)
print(dir(hits_track))
print(dir(hits_track[1]))

[Track with 5 hits: [#255 module 8 {-25.725, -16.763, -136.918}, #310 module 10 {-15.865, -10.325, -63.08}, #371 module 12 {-12.52, -8.147, -38.08}, #427 module 14 {-9.156, -5.988, -13.08}, #463 module 16 {-5.773, -3.81, 11.919}], Track with 4 hits: [#313 module 10 {-3.351, -37.915, -61.918}, #379 module 12 {-2.479, -29.466, -36.918}, #416 module 14 {-1.87, -21.895, -13.08}, #460 module 16 {-1.174, -13.893, 11.919}], Track with 3 hits: [#526 module 22 {-8.613, 7.776, 88.081}, #565 module 24 {-14.894, 13.435, 111.919}, #602 module 26 {-21.408, 19.405, 136.919}], Track with 5 hits: [#516 module 21 {2.431, -4.936, 75.581}, #540 module 23 {5.386, -10.926, 100.581}, #591 module 26 {9.718, -19.561, 136.919}, #626 module 28 {12.68, -25.531, 161.919}, #663 module 30 {15.675, -31.481, 186.919}], Track with 5 hits: [#875 module 42 {-4.178, -4.705, 486.919}, #903 module 44 {-5.15, -5.781, 586.919}, #927 module 46 {-5.656, -6.338, 636.919}, #952 module 48 {-6.123, -6.883, 686.919}, #973 module 50 

## By track

### Step 1: extract the tracks that contains hits in odd and even modules

In [8]:
odd_counter = 0
even_counter = 0
even_odd_counter = 0

odd_track_list = []
even_track_list= []
even_odd_track_list = []

for track in tracks:
    hits = track.hits
    module_numbers = [hit.module_number for hit in hits]
    #print(module_numbers)

    if all(num % 2 == 1 for num in module_numbers):
        odd_counter += 1
        odd_track_list.append(track)

    # check if all numbers are even
    elif all(num % 2 == 0 for num in module_numbers):
        even_counter += 1
        even_track_list.append(track)

    # otherwise, some numbers are odd and some are even
    else:
        even_odd_track_list.append(track)
        even_odd_counter += 1
    

print(odd_counter, even_counter, even_odd_counter)
print(odd_counter+ even_counter+ even_odd_counter)
print(even_odd_counter/(odd_counter+ even_counter+ even_odd_counter))
print(len(even_odd_track_list))


73 61 19
153
0.12418300653594772
19


### Step 2: taking into account the fact the hits have to happen into consecutive modules

In [9]:
def is_consecutive_odd(list):
    if all(x % 2 == 1 for x in list):
        for i in range(len(list) - 1):
            if list[i] + 2 != list[i+1]:
                return False
        return True
    return False

In [10]:
def is_consecutive_even(list):
    if all(x % 2 == 0 for x in list):
        for i in range(len(list) - 1):
            if list[i] + 2 != list[i+1]:
                return False
        return True
    return False

In [12]:
odd_cons_counter = 0
even_cons_counter = 0
even_odd_notcons_counter = 0

odd_cons_track_list = []
even_cons_track_list= []
even_odd_notcons_track_list = []

for track in tracks:
    hits = track.hits
    module_numbers = [hit.module_number for hit in hits]

    if is_consecutive_odd(module_numbers) == True:
        odd_cons_counter += 1
        odd_cons_track_list.append(track)

    elif is_consecutive_even(module_numbers) == True:
        even_cons_counter += 1
        even_cons_track_list.append(track)

    else:
        even_odd_notcons_counter += 1
        even_odd_notcons_track_list.append(track)
    
print(odd_cons_counter, even_cons_counter, even_odd_notcons_counter)
print(odd_cons_counter+ even_cons_counter + even_odd_notcons_counter)
print(even_odd_notcons_counter/(odd_cons_counter+ even_cons_counter+ even_odd_notcons_counter))

64 58 31
153
0.20261437908496732


### Step 3: analysis of the possibility of reconstructible tracks in the even_odd_notcons tracks

Criteria for a track to be considered as constructible by the current algorithm: need to have at least 3 hits in consecutive modules either odd or even 

In [13]:
def is_consecutive(lst):
    # Check for consecutive even numbers
    for i in range(len(lst) - 2):
        if all(num % 2 == 0 for num in lst[i:i+3]) and lst[i+2] - lst[i] == 4:
            return True
    # Check for consecutive odd numbers
    for i in range(len(lst) - 2):
        if all(num % 2 == 1 for num in lst[i:i+3]) and lst[i+2] - lst[i] == 4:
            return True
    return False


In [14]:
cons_counter = 0
notcons_counter = 0
cons_track_list = []
notcons_track_list = []

for track in  even_odd_notcons_track_list:
    hits = track.hits
    module_numbers = [hit.module_number for hit in hits]

    if is_consecutive(module_numbers) == True:
        cons_counter += 1
        cons_track_list.append(track)

    else:
        notcons_counter += 1
        notcons_track_list.append(track)
    
print(cons_counter, notcons_counter)
print(notcons_track_list)

27 4
[Track with 3 hits: [#56 module 1 {37.543, -6.861, -275.58}, #171 module 5 {28.779, -13.836, -225.58}, #232 module 7 {21.954, -9.37, -200.58}], Track with 3 hits: [#331 module 11 {-8.848, 19.601, -50.58}, #397 module 13 {-6.107, 13.515, -25.58}, #429 module 14 {-4.569, 10.188, -11.918}], Track with 3 hits: [#307 module 10 {10.658, -24.947, -63.08}, #374 module 12 {6.866, -15.905, -38.08}, #409 module 13 {4.758, -11.023, -24.418}], Track with 4 hits: [#494 module 18 {-5.346, 8.243, 38.081}, #510 module 20 {-9.586, 14.816, 63.081}, #535 module 23 {-15.868, 24.443, 99.419}, #567 module 25 {-20.153, 31.035, 124.419}]]


In [15]:
print(odd_cons_counter, even_cons_counter, cons_counter, notcons_counter)
print(odd_cons_counter+ even_cons_counter+ even_odd_notcons_counter)
print(even_odd_notcons_counter/(odd_cons_counter+ even_cons_counter+ even_odd_notcons_counter))
print(notcons_counter/(odd_cons_counter+ even_cons_counter+ cons_counter))

64 58 27 4
153
0.20261437908496732
0.026845637583892617


In [16]:
def algo_reconstructible_tracks_stat(tracks):
    n_tracks = 0

    odd_cons_counter = 0
    even_cons_counter = 0
    even_odd_notcons_counter = 0

    odd_cons_track_list = []
    even_cons_track_list= []
    even_odd_notcons_track_list = []

    cons_counter = 0
    notcons_counter = 0
    cons_track_list = []
    notcons_track_list = []

    for track in tracks:
        n_tracks += 1
        hits = track.hits
        module_numbers = [hit.module_number for hit in hits]

        if is_consecutive_odd(module_numbers) == True:
            odd_cons_counter += 1
            odd_cons_track_list.append(track)

        elif is_consecutive_even(module_numbers) == True:
            even_cons_counter += 1
            even_cons_track_list.append(track)

        else:
            even_odd_notcons_counter += 1
            even_odd_notcons_track_list.append(track)
    
    for track in  even_odd_notcons_track_list:
        hits = track.hits
        module_numbers = [hit.module_number for hit in hits]

        if is_consecutive(module_numbers) == True:
            cons_counter += 1
            cons_track_list.append(track)

        else:
            notcons_counter += 1
            notcons_track_list.append(track)


    total_recon_track_odd = odd_cons_counter
    total_recon_track_even = even_cons_counter
    total_recon_track_even_odd = cons_counter
    total_not_recon_track = notcons_counter
    percentage_lost_track_by_event =  total_not_recon_track/n_tracks
    return total_recon_track_odd, total_recon_track_even, total_recon_track_even_odd, total_not_recon_track, percentage_lost_track_by_event
    

## For all events

In [17]:
def algo_reconstructible_tracks_stat_events(events_list):

    dataset_total_recon_track_odd=0 
    dataset_total_recon_track_even=0
    dataset_total_recon_track_even_odd=0 
    dataset_total_not_recon_track = 0
    dataset_percentage_lost_track_by_event= 0

    for event in events_list:
    
        tracks = event.real_tracks

        #Statistics by event
        total_recon_track_odd, total_recon_track_even, total_recon_track_even_odd, total_not_recon_track, percentage_lost_track_by_event = algo_reconstructible_tracks_stat(tracks)

        #Statistics by dataset
        dataset_total_recon_track_odd = dataset_total_recon_track_odd + total_recon_track_odd
        dataset_total_recon_track_even = dataset_total_recon_track_even + total_recon_track_even
        dataset_total_recon_track_even_odd = dataset_total_recon_track_even_odd + total_recon_track_even_odd
        dataset_total_not_recon_track = dataset_total_not_recon_track + total_not_recon_track
        dataset_percentage_lost_track_by_event = dataset_total_not_recon_track/(dataset_total_recon_track_odd + dataset_total_recon_track_even + dataset_total_recon_track_even_odd)


    return dataset_total_recon_track_odd ,dataset_total_recon_track_even, dataset_total_recon_track_even_odd, dataset_total_not_recon_track, dataset_percentage_lost_track_by_event

### Minibias

In [18]:
algo_reconstructible_tracks_stat_events(all_events_minibias)

(108669, 112978, 42874, 10364, 0.039180254119710725)

### Bsphiphi

In [19]:
all_events_bs,all_modules_bs, all_tracks_bs = load_events_modules_tracks('bsphiphi', 1, False)

Successfully loaded 1000 events from bsphiphi dataset.


In [21]:
algo_reconstructible_tracks_stat_events(all_events_bs)

(130990, 136289, 53068, 12179, 0.03801814906960265)